In [3]:
import os
import cv2
import yaml  # 用來讀取 data.yaml

# 設定原始資料夾
dataset_path = "Convert-Dataset-New-2"
output_path = "dataset-yolo-cls-cropped"

# 設定最小影像尺寸（例如 32x32）
min_size = 32

# 讀取 data.yaml 獲取類別名稱
yaml_path = os.path.join(dataset_path, "data.yaml")
with open(yaml_path, "r", encoding="utf-8") as f:
    data_config = yaml.safe_load(f)

# 取得類別名稱對應
class_names = data_config["names"]  # 這是一個 list，對應類別 ID
num_classes = data_config["nc"]  # 類別數量

# 建立新資料夾
os.makedirs(output_path, exist_ok=True)

# 指定要保留並合併的類別
class_mapping = {
    "front_wheel": "wheel",
    "back_wheel": "wheel",
    "front_handle": "handle",
    "back_handle": "handle",
    "front_pedal": "pedal",
    "back_pedal": "pedal",
    "front_mudguard": "mudguard",
    "back_mudguard": "mudguard",
    "saddle": "saddle",
    "steer": "steer",
    "kickstand": "kickstand",
    "lock": "lock",
    "chain": "chain"
}

# 處理 Train 和 Val
for split in ["train", "valid"]:
    images_dir = os.path.join(dataset_path, split, "images")
    labels_dir = os.path.join(dataset_path, split, "labels")
    
    if not os.path.exists(labels_dir):
        continue  # 如果沒有標註，跳過

    # 讀取標註檔案
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue
        
        label_path = os.path.join(labels_dir, label_file)
        image_name = label_file.replace(".txt", ".jpg")
        image_path = os.path.join(images_dir, image_name)

        if not os.path.exists(image_path):
            continue  # 確保影像存在

        # 讀取影像
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        img_height, img_width, _ = image.shape  # 取得影像尺寸

        # 讀取標註檔案
        with open(label_path, "r") as f:
            lines = f.readlines()

        # 處理每一個 bounding box
        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # 確保資料完整
            
            class_id = int(parts[0])  # 類別 ID
            class_name = class_names[class_id]  # 取得對應的類別名稱

            # **篩選出目標類別，並進行類別合併**
            if class_name not in class_mapping:
                continue  # 跳過不在範圍內的類別
            
            merged_class_name = class_mapping[class_name]  # 轉換成合併後的類別名稱

            x_center, y_center, width, height = map(float, parts[1:])

            # 轉換相對座標為實際像素座標
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)

            # 確保座標不超出範圍
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(img_width, x2), min(img_height, y2)

            # 裁剪影像
            cropped_img = image[y1:y2, x1:x2]

            # **過濾掉過小的影像**
            if cropped_img.shape[0] < min_size or cropped_img.shape[1] < min_size:
                continue  # 忽略尺寸過小的影像
            
            # **使用合併後的類別名稱來建立資料夾**
            class_dir = os.path.join(output_path, split, merged_class_name)  
            os.makedirs(class_dir, exist_ok=True)

            # 儲存裁剪影像
            cropped_filename = f"{image_name.replace('.jpg', '')}_{idx}.jpg"
            cropped_path = os.path.join(class_dir, cropped_filename)
            cv2.imwrite(cropped_path, cropped_img)

# 處理完所有資料後，將 valid 重新命名為 val
valid_path = os.path.join(output_path, "valid")
val_path = os.path.join(output_path, "val")

# 確保 valid 存在，然後重新命名
if os.path.exists(valid_path):
    os.rename(valid_path, val_path)
    print(f"✅ `valid/` 已成功更名為 `val/`")
else:
    print(f"⚠️ `valid/` 目錄不存在，可能已經是 `val/` 或未生成。")

print("✅ YOLO-CLS 裁剪 & 類別合併完成！")

✅ `valid/` 已成功更名為 `val/`
✅ YOLO-CLS 裁剪 & 類別合併完成！


In [4]:
# 設定資料集的根目錄
dataset_root = "dataset-yolo-cls-cropped"

# 設定 train/ 和 val/ 的路徑
train_dir = os.path.join(dataset_root, "train")
val_dir = os.path.join(dataset_root, "val")

# 檢查 train/ 是否存在
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"❌ 錯誤: `{train_dir}` 不存在！請檢查你的資料集路徑。")

# 讀取類別名稱
class_names = sorted(os.listdir(train_dir))  # 取得類別名稱
num_classes = len(class_names)  # 計算類別數量

# 建立 YAML 內容
yaml_content = {
    "train": os.path.abspath(train_dir),  # 設定 train 資料夾的完整路徑
    "val": os.path.abspath(val_dir),      # 設定 val 資料夾的完整路徑
    "nc": num_classes,                    # 類別數量
    "names": class_names                   # 類別名稱
}

# 儲存 `data.yaml`
yaml_path = os.path.join(dataset_root, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.dump(yaml_content, f, default_flow_style=False, allow_unicode=True)

print(f"✅ `data.yaml` 已成功建立，路徑: {yaml_path}")

✅ `data.yaml` 已成功建立，路徑: dataset-yolo-cls-cropped\data.yaml


In [5]:
from ultralytics import YOLO

# 載入 YOLOv8 分類模型
model = YOLO("yolov8n-cls.pt")

model.train(data="./dataset-yolo-cls-cropped", epochs=20, imgsz=224, batch=32)

New https://pypi.org/project/ultralytics/8.3.84 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.81  Python-3.9.21 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=./dataset-yolo-cls-cropped, epochs=20, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

100%|██████████| 5.35M/5.35M [00:04<00:00, 1.30MB/s]


AMP: checks passed 


train: Scanning C:\Users\alex9\Desktop\Surveying_Camp\dataset-yolo-cls-cropped\train... 70929 images, 0 corrupt: 100%|██████████| 70929/70929 [01:22<00:00, 864.27it/s] 


train: New cache created: C:\Users\alex9\Desktop\Surveying_Camp\dataset-yolo-cls-cropped\train.cache


...

               classes   top1_acc   top5_acc: 100%|██████████| 159/159 [00:09<00:00, 16.66it/s]

                   all      0.892      0.995



      Epoch    GPU_mem       loss  Instances       Size


       2/20     0.359G     0.3868         32        224:  67%|██████▋   | 1488/2217 [01:38<00:48, 15.13it/s]


KeyboardInterrupt: 

In [2]:
import os
import yaml
import shutil  # 搬移檔案
import pandas as pd  # 儲存 CSV
from ultralytics import YOLO

# 載入訓練好的 YOLOv8 分類模型
final_model = YOLO("./runs/classify/train/weights/last.pt")

# 設定輸入影像資料夾
input_folder = "./KF07"  # 你的影像資料夾

# 設定輸出結果資料夾 (依據 `input_folder` 命名)
output_root = f"{input_folder}_result"
saddle_output_folder = os.path.join(output_root, "saddle_output")  # ✅ 存放高機率 "saddle" 影像
not_saddle_output_folder = os.path.join(output_root, "not_saddle_output")  # ✅ 存放非 "saddle" 或低機率 saddle 影像
os.makedirs(saddle_output_folder, exist_ok=True)
os.makedirs(not_saddle_output_folder, exist_ok=True)

# 讀取 `data.yaml` 獲取類別名稱
yaml_path = "./dataset-yolo-cls-cropped/data.yaml"  # 你的 data.yaml 路徑
with open(yaml_path, "r", encoding="utf-8") as f:
    data_config = yaml.safe_load(f)

# 取得類別名稱對應
class_names = data_config["names"]  # 這是一個 list，對應類別 ID
saddle_class_id = class_names.index("saddle")  # 找到 "saddle" 在 class_names 裡的索引

# 用來存放 `saddle` 和 `not_saddle` 影像的辨識結果
saddle_results = []
not_saddle_results = []

# 進行影像分類
results = final_model.predict(source=input_folder, save=False)  # 不再存 `cls_output`

for result in results:
    # 取得該影像的分類機率
    probs = result.probs  # 取得所有類別的機率

    if probs is not None:  # 確保有機率值
        max_prob_idx = probs.top1  # 取得最高機率的類別索引
        max_prob_value = probs.top1conf  # 取得最高機率值
        image_path = result.path  # 影像路徑
        image_name = os.path.basename(image_path)  # 取得檔案名稱
        
        print(f"📌 影像: {image_name}")
        print(f"🔹 預測類別: {class_names[max_prob_idx]}")
        print(f"🔹 最高機率: {max_prob_value:.4f}\n")

        # ✅ **分類為 `saddle` 且機率 > 0.85**
        if max_prob_idx == saddle_class_id and max_prob_value > 0.85:
            shutil.copy(image_path, os.path.join(saddle_output_folder, image_name))
            saddle_results.append([image_name, max_prob_value])  

        # ✅ **非 "saddle" 或 "saddle" 但機率 ≤ 0.85**
        else:
            shutil.copy(image_path, os.path.join(not_saddle_output_folder, image_name))
            not_saddle_results.append([image_name, max_prob_value, class_names[max_prob_idx]])  

# **儲存 `saddle` 結果為 CSV (只包含機率 > 0.85 的影像)**
saddle_csv_path = os.path.join(saddle_output_folder, "saddle_results.csv")
df_saddle = pd.DataFrame(saddle_results, columns=["Image", "Probability"])
df_saddle.to_csv(saddle_csv_path, index=False)

# **儲存 `not_saddle` 結果為 CSV**
not_saddle_csv_path = os.path.join(not_saddle_output_folder, "not_saddle_results.csv")
df_not_saddle = pd.DataFrame(not_saddle_results, columns=["Image", "Probability", "Predicted_Class"])
df_not_saddle.to_csv(not_saddle_csv_path, index=False)

print(f"✅ `saddle_output` 內含: {os.listdir(saddle_output_folder)}")
print(f"✅ `not_saddle_output` 內含: {os.listdir(not_saddle_output_folder)}")
print(f"📄 `saddle_results.csv` (機率 > 0.85) 已存入: {saddle_csv_path}")
print(f"📄 `not_saddle_results.csv` 已存入: {not_saddle_csv_path}")


image 1/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_0.jpg: 224x224 steer 0.32, handle 0.27, saddle 0.19, mudguard 0.12, wheel 0.07, 4.4ms
image 2/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_1.jpg: 224x224 saddle 0.52, wheel 0.29, mudguard 0.05, steer 0.04, pedal 0.04, 4.6ms
image 3/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_10.jpg: 224x224 steer 0.24, wheel 0.17, pedal 0.14, handle 0.11, mudguard 0.10, 7.2ms
image 4/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_11.jpg: 224x224 steer 0.40, handle 0.26, chain 0.13, kickstand 0.09, saddle 0.08, 6.2ms
image 5/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_12.jpg: 224x224 wheel 0.54, mudguard 0.44, pedal 0.01, lock 0.00, saddle 0.00, 6.3ms
image 6/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_13.jpg: 224x224 steer 0.28, handle 0.26, wheel 0.20, saddle 0.15, mudguard 0.10, 3.7ms
image 7/90 c:\Users\alex9\Desktop\Surveying_Camp\KF07\mask_14.jpg: 224x224 handle 0.60, steer 0.17, saddle 0.07, chain 0.06, kic